In [1]:
import yaml
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from INEGIpy import Indicadores
from sie_banxico import SIEBanxico

In [2]:
def cargar_config(ruta_archivo='tokens.yaml'):
    """
    Carga los tokens de APIs desde un archivo YAML de configuración.
    
    Args:
        ruta_archivo (str): Ruta al archivo YAML (por defecto 'tokens.yaml')
        
    Returns:
        dict: Diccionario con la configuración cargada
    """
    try:
        with open(ruta_archivo, 'r') as archivo:
            config = yaml.safe_load(archivo)
        print("✅ Configuración cargada exitosamente")
        return config
    except FileNotFoundError:
        print(f"❌ Error: No se encontró el archivo {ruta_archivo}")
        print(f"   Crea este archivo con la estructura adecuada para los tokens")
        return None
    except yaml.YAMLError as e:
        print(f"❌ Error al parsear el archivo YAML: {e}")
        return None

In [3]:
def obtener_inflacion_inegi(token_inegi):
    """
    Obtiene datos de inflación (INPC) del INEGI utilizando directamente la clase Indicadores.
    
    Esta función consulta específicamente el indicador 628465 que corresponde al:
    "Índice Nacional de Precios al Consumidor (INPC) - Cambio porcentual anual"
    
    Args:
        token_inegi (str): Token de acceso para la API de INEGI
        
    Returns:
        DataFrame: DataFrame con los datos de inflación con columnas 'fecha' e 'inflacion_anual'
    """
    try:
        # Inicializamos directamente la clase Indicadores con el token
        inegi_indicadores = Indicadores(token=token_inegi)
        
        # Usamos el método obtener_df de la clase Indicadores
        # Indicador 628465: INPC mensual, cambio porcentual anual
        df_inflacion = inegi_indicadores.obtener_df(
            indicadores='628465',
            nombres='inflacion_anual',
            inicio=(datetime.now() - timedelta(days=365*3)).strftime('%Y-%m-%d'),
            banco='BIE'  # Especificamos explícitamente el banco
        )
        
        # Preparamos el dataframe para análisis posterior
        df_inflacion = df_inflacion.reset_index()
        df_inflacion.rename(columns={'index': 'fecha'}, inplace=True)
        
        print(f"✅ Datos de inflación obtenidos: {len(df_inflacion)} registros")
        print(f"   Período: {df_inflacion['fecha'].min().strftime('%Y-%m-%d')} a {df_inflacion['fecha'].max().strftime('%Y-%m-%d')}")
        
        return df_inflacion
    
    except Exception as e:
        print(f"❌ Error al obtener datos de inflación del INEGI: {e}")
        print("   Verifica el token y la conexión a internet.")
        return None


In [26]:
def obtener_tasa_interes_banxico(token, fecha_inicio=None, fecha_fin=None):
    """
    Obtiene datos de la TIIE a 28 días directamente desde Banxico.
    
    Esta función consulta específicamente la serie SF43783 que corresponde a:
    "Tasas de Interés Interbancarias Semanales Por ciento anual TIIE a 28 días"
    
    Args:
        token (str): Token de la API de Banxico
        fecha_inicio (str, optional): Fecha de inicio en formato 'YYYY-MM-DD'
        fecha_fin (str, optional): Fecha final en formato 'YYYY-MM-DD'
        
    Returns:
        DataFrame: DataFrame con los datos de tasas de interés con columnas 'fecha' y 'tiie_28'
    """
    # Si no se especifican fechas, tomamos los últimos dos años
    if not fecha_inicio:
        fecha_inicio = (datetime.now() - timedelta(days=365*2)).strftime('%Y-%m-%d')
    if not fecha_fin:
        fecha_fin = datetime.now().strftime('%Y-%m-%d')
    
    try:
        # Importamos la biblioteca SIEBanxico
        from sie_banxico import SIEBanxico
        
        # Inicializamos el cliente con el token y la serie SF43783 (Tasa de CETES 28 días)
        # Importante: La serie debe pasarse como una lista, incluso si solo consultamos una
        banxico = SIEBanxico(token=token, id_series=['SF43783'], language='es')
        
        # Obtenemos la serie en el rango de fechas especificado
        # Usamos get_timeseries_range para obtener solo los datos dentro del período solicitado
        respuesta = banxico.get_timeseries_range(init_date=fecha_inicio, end_date=fecha_fin)
        
        # Extraemos los datos de la respuesta JSON
        datos = respuesta['bmx']['series'][0]['datos']
        
        # Convertimos a DataFrame
        df = pd.DataFrame(datos)
        
        # Procesamos el DataFrame
        df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')
        df['dato'] = df['dato'].str.replace(',', '').astype(float)
        
        # Renombramos la columna de valores
        df = df.rename(columns={'dato': 'tiie_28'})
        
        print(f"✅ Datos de tasa de interés TIIE 28 días (SF43783) obtenidos: {len(df)} registros")
        print(f"   Período: {df['fecha'].min().strftime('%Y-%m-%d')} a {df['fecha'].max().strftime('%Y-%m-%d')}")
        
        return df
    
    except ImportError:
        print("❌ Error: No se encontró la biblioteca sie_banxico.")
        print("   Por favor, instálala con: pip install sie_banxico")
        return None
    except Exception as e:
        print(f"❌ Error al obtener datos de Banxico: {e}")
        print("   Verifica que el token sea válido y que tengas conexión a internet.")
        return None

In [27]:
def explorar_indicadores_inegi(token_inegi, busqueda=None, banco='BIE'):
    """
    Explora el catálogo de indicadores disponibles en INEGI utilizando directamente la clase Indicadores.
    Opcionalmente filtra por términos de búsqueda.
    
    Args:
        token_inegi (str): Token de acceso para la API de INEGI
        busqueda (str, optional): Término para filtrar indicadores
        banco (str): Banco de indicadores a consultar ('BIE' o 'BISE')
        
    Returns:
        DataFrame: DataFrame con los indicadores encontrados
    """
    try:
        # Inicializamos directamente la clase Indicadores con el token
        inegi_indicadores = Indicadores(token=token_inegi)
        
        # Obtenemos el catálogo completo de indicadores
        indicadores = inegi_indicadores.catalogo_indicadores(banco=banco)
        
        # Si se proporciona un término de búsqueda, filtramos
        if busqueda:
            indicadores_filtrados = indicadores[
                indicadores['descripcion'].str.contains(busqueda, case=False, regex=True)
            ]
            print(f"✅ Se encontraron {len(indicadores_filtrados)} indicadores relacionados con '{busqueda}'")
            return indicadores_filtrados
        
        print(f"✅ Se encontraron {len(indicadores)} indicadores en el banco {banco}")
        return indicadores
        
    except Exception as e:
        print(f"❌ Error al consultar el catálogo de indicadores: {e}")
        return None

In [30]:
# Cargamos la configuración 
config = cargar_config()
if not (config and 'banxico' in config and 'inegi' in config):
    print("❌ Error: Configuración incompleta. Verifica el archivo config.yaml")


# Obtenemos tokens 
token_banxico = config['banxico']['token']
token_inegi = config['inegi']['token']
inegi_indicadores = Indicadores(token=token_inegi)

✅ Configuración cargada exitosamente


In [23]:
df_inflacion, metadatos = inegi_indicadores.obtener_df(
    indicadores='910399',
    nombres='inflacion_mensual',
    clave_area = '01',
    banco='BIE', 
    metadatos = True  # Especificamos explícitamente el banco
)

/opt/homebrew/Caskroom/miniconda/base/envs/analisis-economico/lib/python3.11/site-packages/INEGIpy/_indicadores.py:112: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.set_index(pd.to_datetime(df.fechas),inplace=True, drop=True)


In [31]:
df_inflacion.head()

,inflacion_mensual
fechas,
1969-01-01,NaN
1969-02-01,0.36
1969-03-01,0.10
1969-04-01,0.27
1969-05-01,0.00


In [32]:
banxico = SIEBanxico(token=token_banxico, id_series=['SF43783'], language='es')
respuesta = banxico.get_timeseries_range(init_date='2000-12-01', end_date='2020-12-01')

In [33]:
respuesta

{'bmx': {'series': [{'idSerie': 'SF43783',
    'titulo': 'TIIE a 28 días Tasa de interés en por ciento anual',
    'datos': [{'fecha': '04/12/2000', 'dato': '18.8000'},
     {'fecha': '05/12/2000', 'dato': '18.7450'},
     {'fecha': '06/12/2000', 'dato': '18.6350'},
     {'fecha': '07/12/2000', 'dato': '18.3550'},
     {'fecha': '08/12/2000', 'dato': '18.2500'},
     {'fecha': '11/12/2000', 'dato': '18.1200'},
     {'fecha': '13/12/2000', 'dato': '18.0350'},
     {'fecha': '14/12/2000', 'dato': '18.0200'},
     {'fecha': '15/12/2000', 'dato': '18.0000'},
     {'fecha': '18/12/2000', 'dato': '17.9700'},
     {'fecha': '19/12/2000', 'dato': '17.9250'},
     {'fecha': '20/12/2000', 'dato': '17.9200'},
     {'fecha': '21/12/2000', 'dato': '18.5750'},
     {'fecha': '22/12/2000', 'dato': '18.6500'},
     {'fecha': '26/12/2000', 'dato': '18.7950'},
     {'fecha': '27/12/2000', 'dato': '18.7225'},
     {'fecha': '28/12/2000', 'dato': '18.7200'},
     {'fecha': '29/12/2000', 'dato': '18.8550'}